# Data Visualization (2017/18)

## Assignment 5 - Visual Analysis of Graphs

Presented by Group ??: 
- Name1
- Name2

Date: xx.01.2018

Hints: 


In [1]:
import numpy as np
import scipy as sp
from scipy import interpolate

from bokeh.plotting import figure, output_notebook, show
output_notebook()

import networkx as nx

from bokeh.io import show, output_file
from bokeh.plotting import figure
from bokeh.models import Oval, Ellipse, Circle, Text, Line, MultiLine, Arrow, NormalHead, VeeHead, Label, ColumnDataSource
from bokeh.models import GraphRenderer, GlyphRenderer, StaticLayoutProvider, HoverTool, TapTool, BoxSelectTool, Range1d
from bokeh.models.graphs import from_networkx, NodesAndLinkedEdges, EdgesAndLinkedNodes

Loading BokehJS ...

# Exercise 1: Getting started with graphs in bokeh

Bokeh recently started to provide graph rendering support: https://bokeh.pydata.org/en/latest/docs/user_guide/graph.html#

This first exercise shall help you work with the required packages.

To support graph analysis, Bokeh has a networkx integration. Networkx https://networkx.github.io is a python package to work with complex networks (creation, manipulation, IO, analysis, layout). Networkx is very strong with algorithmic analysis of graphs, but provides only a few basic layouts that we will test first. In the next part we will extend the code and switch to the more powerful layouts from the GraphViz library https://www.graphviz.org, provided as python package pygraphviz https://pygraphviz.github.io.

**<font color="deeppink">Task 1a)</font>**: You are given a triary tree and bokeh code for rendering below. Render the tree using the networkx native layout algorithms. Check all algorithms in the list below (using &#9746;) that are helpful in analyzing the given tree. Give a short reason for your choice.

- &#9744; **bipartite_layout**: your reason here
- &#9744; **circular_layout**: 
- &#9744; **kamada_kawai_layout**: 
- &#9744; **random_layout**: 
- &#9744; **shell_layout**: 
- &#9744; **spring_layout**: 
- &#9744; **spectral_layout**: 

Hints:
- Networkx integration in bokeh: https://bokeh.pydata.org/en/latest/docs/user_guide/graph.html#
- Documentation for layouts: https://networkx.github.io/documentation/stable/reference/drawing.html#module-networkx.drawing.layout 
- Do not use other libraries yet!

In [2]:
plot = figure( title="Networkx Integration Demonstration", x_range=(-1,1), y_range=(-1,1), 
               plot_width=300, plot_height=300 )

G = nx.balanced_tree( 3, 3 )

# how to pass parameters to the networkx algorithms:
# graph = from_networkx(G, nx.bipartite_layout, nodes=[0], align='horizontal', scale=1, center=(0,0))

graph = from_networkx(G, nx.circular_layout, scale=1, center=(0,0))

plot.renderers.append(graph)

show(plot)

As you may have noticed, none of the algorithms provided a classical hierarchical tree layout. We will work towards this now and style the graph. We will now use layout algorithms from the graphviz library - a widely used graph drawing library with very good layout algorithms - probided through pygraphviz https://pygraphviz.github.io.

Your goal is to obtain the following layout for the triary tree:
<img src="tree.png" width="400pt" />

**<font color="deeppink">Task 1b)</font>**: The code below demonstrates how to use a graphviz layout in bokeh [1] (using networkx [2] and its access to pygraphviz [3]). To obtain the desired layout, update the following three parameters:
- **Choose the correct layout**: The `prog` parameter specifies the layout routine. Select a fitting layout.
- **Distances settings**: The current layout has a poor aspect ratio. Adjust the distance settings in the `args` to improve aspect ratio and make the tree fit in the given x-/y-range. Documentation: https://graphviz.gitlab.io/_pages/pdf/dot.1.pdf
- **Change style**: The glyphs for the nodes and edges are defined in the bokeh part by means of renderers. See the examples from bokeh [1] and change the shape of the glyph (draw larger circles in a new color, e.g., orange) as well as the color and line style of the edges.


<font size="1">[1] https://bokeh.pydata.org/en/latest/docs/user_guide/graph.html#<br>
[2] https://networkx.github.io/documentation/stable/reference/generated/networkx.drawing.nx_agraph.graphviz_layout.html#networkx.drawing.nx_agraph.graphviz_layout<br>
[3] http://pygraphviz.github.io/documentation/latest/reference/agraph.html</font>

In [3]:
plot = figure(title="Tree layout with graphviz", x_range=(-20,250), y_range=(-20,250) )

graph = from_networkx( G, nx.nx_agraph.graphviz_layout, prog='neato',
                       args="-Gnodesep=1 -Granksep=1 -Nwidth=1 -Nheight=1 -Nlabel=\"\"")

graph.node_renderer.glyph = Circle()
graph.edge_renderer.glyph = MultiLine()

plot.renderers.append( graph )

show(plot)

# Exercise 2: Visualizing hierarchies

In this exercise, we will extend the bokeh code to render hierarchical graphs with directed, curved edges and labels as depicted in Figure 1. 

**Figure 1**: Hierarchy G rendered using dot command line tool. Command line code to recreate the image (you have to have graphviz installed): `dot -Tpdf tree.gv -o tree.pdf`

![title](hierarchy.png)


Below you are given starter code which does the following things:
- Compute the dot layout for a given graph.
- Extract layout information for nodes from the dot algorithm and provide it to bokeh (see more details below).
- Render edges as B-splines.



In [4]:
def bSplineInterpolation( p ):
    ctr =np.array(p)
    x=ctr[:,0]
    y=ctr[:,1]
    l=len(x)
    
    t=np.linspace(0,1,l-2,endpoint=True)
    t=np.append([0,0,0],t)
    t=np.append(t,[1,1,1])
    tck=[t,[x,y],3]

    u3=np.linspace(0,1,(max(l*2,20)),endpoint=True)
    return sp.interpolate.splev(u3,tck) 

def getPos( s ):
    x,y = s.split(',',1)
    return (float(x),float(y))


In [5]:
def directedGraphLayout( graph ):
    A = nx.nx_agraph.to_agraph(graph)
    A.layout( prog='dot' )

    # set the nodes end edges
    nodes = list(A.nodes())
    edges = list(A.edges())

    edges_start = [edge[0] for edge in edges]
    edges_end = [edge[1] for edge in edges]

    graph_renderer = GraphRenderer()

    graph_renderer.node_renderer.data_source.data = dict(
        index=nodes,
        width=[65*float(i.attr['width']) for i in A.nodes()],
        height=[65*float(i.attr['height']) for i in A.nodes()]
    )
    graph_renderer.edge_renderer.data_source.data = dict(
        start=edges_start,
        end=edges_end
    )

    # create a layout with node positions
    graph_layout = {}
    for n in A.nodes():
        graph_layout[n] = tuple(map(float, A.get_node(n).attr['pos'].split(',')))
       
    graph_renderer.layout_provider = StaticLayoutProvider(graph_layout=graph_layout)
    
    # create the B-Spline edges
    xs, ys = [], []
    for i in A.edges_iter():
        l = i.attr['pos'].split(' ')
        p = [ getPos(s) for s in l[1:]] + [getPos(l[0][2:])]
        out = bSplineInterpolation(p)
        xs.append( out[0].tolist() )
        ys.append( out[1].tolist() )
    
    graph_renderer.edge_renderer.data_source.data['xs'] = xs
    graph_renderer.edge_renderer.data_source.data['ys'] = ys
    
    return graph_renderer

The function `directedGraphLayout( G )` returns a Bokeh `GraphRenderer` object for  which you have access to the following information:
- **Nodes**: 
 - width and height (`graph_renderer.node_renderer.data_source`)
 - label and position. Iterate over nodes and their positions using the following loop<br>
 `for label,pos in graph.layout_provider.graph_layout.items():`
 - position
- **Edges**: 
 - Sampled B-spline positions (xs and ys in `graph_renderer.edge_renderer.data_source`). Iteration:<br>
 `for x,y in zip(graph.edge_renderer.data_source.data['xs'],graph.edge_renderer.data_source.data['ys']):`
 - The last two vertices are the start and end of the arrow tip still to be drawn.

**Hints**:
 - `print(G)` for networkx graph objects
 - `graph.pprint()` for GraphRenderer

**<font color="deeppink">Task</font>**: Below you are given a simple hierarchy G and the bokeh code to render it using the routines from above. Edit the rendering code to make G look as similar as possible to the output from the command line dot algorithm in figure 1.

ToDo:
- Adjust the size and shape of the glyphs (use Ellipse glyph to have correct width/height values).
- Render the text using Bokeh's labels (change font, font size, position).
- Add tips to the edges (you'll have to use annotations, lines with arrows do not work yet).

In [6]:
G = nx.DiGraph()
G.add_edge('a','aa')
G.add_edge('a','ab')
G.add_edge('a','bbc')
G.add_edge('b','ab')
G.add_edge('b','bb')
G.add_edge('c','bbc')
G.add_edge('bb','bba')
G.add_edge('bb','bbc')
A = nx.nx_agraph.to_agraph(G)

In [7]:
graph = directedGraphLayout( G )

p = figure(title="Directed graph layout", x_range=(-50,350), y_range=(-20,190), plot_width=950, 
           plot_height=450 )

p.renderers.append(graph)

p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
  
show(p)

# Exercise 3: Analyzing graphs

In the material folder you'll find two graphs that you shall render using an appropriate technique and answer questions relating to the data.

**<font color="deeppink">Task 3a)</font>**: The file `lesmis.gml` contains a coappearance network of characters in the novel Les Miserables. Display the graph using a node-link-diagram of your choice.

Answer the following questions:
- **Graph type**: Which type of graph do you have and what is your chosen layout?
- **Identify a community**: A community in a graph is a set of nodes that is densely connected. See the example below where gray circles enclose communities:
![Community in graphs](Network_Community_Structure.png)
 Use the routine `greedy_modularity_communities(G)` from networkx to extract communities and color the nodes of each community in the same color. Now answer the following questions:
 - Name the color of a community that matches your expected structure of a community.
 - Name the color of a community that you consider not optimal. Explain problems of this community.
- **Unusual people**: 
 - Find three people that should be members of multiple communities. State their names and list the communities they should belong to.
 - Find a person that cooccurs with many people that do not occur with anyone else.
 - Can you find a person that is always alone on stage? What would their nodes look like?


Data courtesy: D. E. Knuth, The Stanford GraphBase: A Platform for Combinatorial Computing, Addison-Wesley, Reading, MA (1993). <br>
image: By j_ham3 - Own work, CC BY-SA 3.0, https://commons.wikimedia.org/w/index.php?curid=17125894

In [8]:
G = nx.read_gml("lesmis.gml")

**<font color="deeppink">Task 3b)</font>**: The file `programmingLanguages.gml` contains information on the genealogy of some of the more influential or widely used programming languages as discussed by (Scott, 2000). Edges indicate principal influences on design. Many influences, of course, cannot be shown in a single figure. 

The graph also contains the date for each language which indicates the approximate time at which its features became widely known. This can be used for rendering but cannot be realized with the algorithm we have used so far. It is enough to show dependencies here.

Render the graph using an appropriate layout and provide a method to color a list or set of nodes.

Now answer the following questions:
- **Initial languages**: Which languages were not influenced by others? How can they be identified?
- **Long history**: Which language has the longest history, i.e., the longest path to a upper leaf? What is the path length? Solve this problem algorithmically using an appropriate routine from networkx and color the respective nodes.
- **Most influenced**: Which language has the largest number of direct influences? How many?
- **Java influences**: List all languages that influenced Java (directly and indirectly). Solve this question algorithmically using networkx and color the nodes.
- **Showing time**: Each language also contains a date. How could this be incorporated into the visualization? Can you base the new design on the given one? If no, say why. If yes, reason how difficult the adaption would be and where you would have to be careful.

Scott, Michael Lee. Programming language pragmatics. Morgan Kaufmann, 2000.

In [9]:
G = nx.read_gml("programmingLanguages.gml")